<a href="https://colab.research.google.com/github/Leoli04/llms-notebooks/blob/main/pytorch/basic/neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

神经网络
===============

神经网络可以使用 torch.nn 包构建。

现在您已经了解了 autograd ， nn 依赖于 autograd 来定义模型并区分它们。 nn.Module 包含层和返回 output 的方法 forward(input) 。

例如，看看这个对数字图像进行分类的网络:

![convnet](https://pytorch.org/tutorials/_static/img/mnist.png)

这是一个简单的前馈网络。它接受输入，将其逐层输入，最后给出输出。



神经网络的典型训练过程如下:

-   定义具有一些可学习参数（或权重）的神经网络
-   迭代输入数据集
-   通过网络处理输入
-   计算损失（输出距离正确还有多远）
-   将梯度传播回网络参数
-   更新网络的权重，通常使用简单的更新规则:
 `weight = weight - learning_rate * gradient`

定义网络
------------------

让我们定义这个网络：


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, input):
        # Convolution layer C1: 1 input image channel, 6 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a Tensor with size (N, 6, 28, 28), where N is the size of the batch
        c1 = F.relu(self.conv1(input))
        # Subsampling layer S2: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 6, 14, 14) Tensor
        s2 = F.max_pool2d(c1, (2, 2))
        # Convolution layer C3: 6 input channels, 16 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a (N, 16, 10, 10) Tensor
        c3 = F.relu(self.conv2(s2))
        # Subsampling layer S4: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 16, 5, 5) Tensor
        s4 = F.max_pool2d(c3, 2)
        # Flatten operation: purely functional, outputs a (N, 400) Tensor
        s4 = torch.flatten(s4, 1)
        # Fully connected layer F5: (N, 400) Tensor input,
        # and outputs a (N, 120) Tensor, it uses RELU activation function
        f5 = F.relu(self.fc1(s4))
        # Fully connected layer F6: (N, 120) Tensor input,
        # and outputs a (N, 84) Tensor, it uses RELU activation function
        f6 = F.relu(self.fc2(f5))
        # Gaussian layer OUTPUT: (N, 84) Tensor input, and
        # outputs a (N, 10) Tensor
        output = self.fc3(f6)
        return output


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


您只需定义 forward 函数，然后使用 autograd 为您自动定义 backward 函数（计算梯度）。您可以在 forward 函数中使用任何张量运算。

模型的可学习参数由 net.parameters() 返回


In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


让我们尝试随机 32x32 输入。注意：该网络 (LeNet) 的预期输入大小为 32x32。要在 MNIST 数据集上使用此网络，请将数据集中的图像大小调整为 32x32。


In [3]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0075,  0.0251,  0.0323,  0.0477, -0.0540, -0.0502, -0.0344,  0.0977,
         -0.0236,  0.0073]], grad_fn=<AddmmBackward0>)


先清空梯度缓存，然后反向传播随机梯度：

In [4]:
net.zero_grad()
out.backward(torch.randn(1, 10))

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p><code>torch.nn</code> 只支持小批量(mini-batches)数据，也就是输入不能是单个样本，比如对于 nn.Conv2d 接收的输入是一个 4 维张量--nSamples * nChannels * Height * Width 。
所以，如果你输入的是单个样本，需要采用 input.unsqueeze(0) 来扩充一个假的 batch 维度，即从 3 维变为 4 维。</p>
</div>



**回顾:**

:   -   `torch.Tensor` - 支持自动分级操作的多维数组，例如 backward() 。还保存梯度 w.r.t.张量。
    -   `nn.Module` - 神经网络模块。封装参数的便捷方式，带有将参数移动到 GPU、导出、加载等的帮助程序。
    -   `nn.Parameter` - 一种张量，当作为属性分配给 Module 时会自动注册为参数。
    -   `autograd.Function` - 实现自动分级操作的前向和后向定义。每个 Tensor 操作至少创建一个 Function 节点，该节点连接到创建 Tensor 的函数并对其历史记录进行编码。



损失函数
=============

损失函数采用（输出，目标）输入对，并计算一个值来估计输出与目标的距离。

 nn 包下有几种不同的[损失函数](https://pytorch.org/docs/nn.html#loss-functions)。一个简单的损失是： nn.MSELoss ，它计算输出和目标之间的均方误差。

For example:


In [5]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.4960, grad_fn=<MseLossBackward0>)


现在，如果您使用 .grad_fn 属性沿向后方向跟踪 loss ，您将看到如下所示的计算图：:

``` {.sourceCode .sh}
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> flatten -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
```

当我们调用 loss.backward() 时，整个图是微分的。神经网络参数，并且图中所有具有 requires_grad=True 的张量将其 .grad 张量随梯度累积。




In [6]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

反向传播
========
反向传播的实现只需要调用 loss.backward() 即可，当然首先需要清空当前梯度缓存，即.zero_grad() 方法，否则之前的梯度会累加到当前的梯度，这样会影响权值参数的更新。

下面是一个简单的例子，以 conv1 层的偏置参数 bias 在反向传播前后的结果为例：


In [7]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0014, -0.0056, -0.0041, -0.0026,  0.0006, -0.0085])


了解更多有关 torch.nn 库,可以看[这里](https://pytorch.org/docs/nn)

更新权重
==================

采用随机梯度下降(Stochastic Gradient Descent, SGD)方法的最简单的更新权重规则如下：

``` {.sourceCode .python}
weight = weight - learning_rate * gradient
```

按照这个规则，代码实现如下所示：e:

``` {.sourceCode .python}
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
```

但是这只是最简单的规则，深度学习有很多的优化算法，不仅仅是 SGD，还有 Nesterov-SGD, Adam, RMSProp 等等，为了采用这些不同的方法，这里采用 torch.optim 库，使用例子如下所示：:

``` {.sourceCode .python}
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update
```


<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p>需要调用 optimizer.zero_grad() 方法清空梯度缓存。</p>
</div>
